In [53]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import csv; import requests
import time
import pandas as pd

In [54]:
url = "https://www.kfcmongolia.com/index"

In [55]:
#CSS Selectors
pick_up_css = "#app > div.kfc_view_box > div > div.__start_ordering_container > div > ul > li:nth-child(1) > a"
drop_down_css = "#delivery > div > form > div > div > span > span > i"
store_select_css = "html body.overflow div.el-select-dropdown.el-popper div.el-scrollbar div.el-select-dropdown__wrap.el-scrollbar__wrap ul.el-scrollbar__view.el-select-dropdown__list"
drop_list_css = ".el-scrollbar__view"
address_confirm_css = "html body.overflow div#app.__ishome div.kfc_view_box div.kfc_dialog div.__start_ordering_container div.__ordering_content div.tab-content div#delivery.tab-pane.active div.__pick_delivery_tab button.kfc_btn.__brand.__address_confirm_button"

#XPATH
store_list_xpath = '/html/body/div[2]/div[1]/div[1]/ul'

path = os.getcwd() + "chromedriver.exe"

In [60]:
def get_storelist(url):
    driver = webdriver.Chrome('D:\projects\webscrap\chromedriver')
    driver.get(url)
    
    #First button to navigate to store slection
    pick_up = driver.find_element(By.CSS_SELECTOR, pick_up_css)
    pick_up.click()
    
    #Drop down button to get the store lists visible
    drop_down = driver.find_element(By.CSS_SELECTOR, drop_down_css)
    drop_down.click()
    
    time.sleep(2)
    #Get the store list
    global store_list
    stores = driver.find_element(By.XPATH, store_list_xpath).text
    store_list = stores.split("\n")
    driver.quit()
    
    return store_list

get_storelist(url)

['']

In [58]:
#Create store num & dictionary from position in the list
store_index = []
for store in store_list:
    store_index.append(store_list.index(store)+1)
store_dict = dict(zip(store_index, store_list))

In [59]:
print("All KFC Stores:")
for key, value in store_dict.items():
    print(f"{key}: {value}")

All KFC Stores:
1: 


In [33]:
prod_df = pd.DataFrame()

In [34]:
#Function to append to a DF for each page
def get_store_items():
    
    sets = driver.find_elements(By.CLASS_NAME,'__main_content_section')
    
    for set in sets:
        set_name = set.find_element(By.TAG_NAME, 'h3').text
        products = set.find_elements(By.CLASS_NAME,'__product_item')

        print(set_name)
        for product in products:
            if product.text != '':
                
                prod_name = product.find_element(By.CLASS_NAME,
                            '__product_im_desc_cont').get_attribute('innerHTML')
                prod_name = prod_name[prod_name.find('"">') + 3: prod_name.find('</p>')]
                print(f"Prod Name:{prod_name}")
                
                prod_img = product.find_element(By.CLASS_NAME,
                            '__product_im_img').get_attribute("innerHTML")
                prod_img = prod_img[prod_img.find('src="')+5:prod_img.find('" alt=')]
                print(f"Image Link:{prod_img}")

                
                #Convert string to integers for KCAL and Price:
                #1: KCAL
                prod_kcal = product.find_element(By.CLASS_NAME,
                            '__product_item_ccalory').text
                prod_kcal_num = []
                
                #2: Price
                prod_price = product.find_element(By.CLASS_NAME,
                            '__product_desc_price').text
                print(f"Prod price:{prod_price}")
                
                #Create a Dictionary and append to a DF
                prod_dict = {'Store': store, 
                             'Set': set_name, 
                             'Prod_name':prod_name, 
                             'Prod_img': prod_img, 
                             'Prod_kcal': prod_kcal,
                             'Prod_price_mnt': prod_price}
                global prod_df
                
                prod_df = prod_df.append(prod_dict, ignore_index = True)
                
                
    return prod_df

In [49]:
prod_df

""


In [31]:
#Initialize the website, then recursively go through all stores.

driver = webdriver.Chrome('D:\projects\webscrap\chromedriver')
driver.get(url)

#Create an F String for store selector 
#drop down, the xpath was indexed.
store_select_xpath = f"/html/body/div[2]/div[1]/div[1]/ul/li[1]"


#First button to navigate to store slection
pick_up = driver.find_element(By.CSS_SELECTOR, pick_up_css)
pick_up.click()

#Drop down button to get the store lists visible
drop_down = driver.find_element(By.CSS_SELECTOR, drop_down_css)
drop_down.click()

#Select specific store based on index
store_select = WebDriverWait(driver,20).until(EC.presence_of_element_located((
    By.XPATH, store_select_xpath )))
store_select.click()

#Load the first page
address_confirm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((
    By.CSS_SELECTOR, address_confirm_css)))
address_confirm.click()

time.sleep(5)

#Navigate to all N stores recursively
for store in store_index:
    
    WebDriverWait(driver,20).until(EC.presence_of_element_located((
    By.CSS_SELECTOR, 
    '#app > div:nth-child(1) > header > div > div > div > div.__header_item.__header_layout_menu > div.__header_address' ))).click()

    store_select_xpath = f"/html/body/div[2]/div[1]/div[1]/ul/li[{store}]/span"
    
    pick_up = driver.find_element(By.CSS_SELECTOR, pick_up_css)
    pick_up.click()

    drop_down = driver.find_element(By.CSS_SELECTOR, drop_down_css)
    drop_down.click()
    
    store_select = WebDriverWait(driver,20).until(EC.presence_of_element_located((
        By.XPATH, store_select_xpath )))
    print(store_select_xpath)
    store_select.click()
    
    address_confirm = WebDriverWait(driver, 20).until(EC.presence_of_element_located((
    By.CSS_SELECTOR, address_confirm_css)))
    
    address_confirm.click()
    
    time.sleep(2)
    
    get_store_items()

driver.quit()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=109.0.5414.120)
